In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
import os
import gc

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedKFold

# model
from sklearn.ensemble import RandomForestRegressor

SEED=42

In [0]:
path = 'drive/My Drive/bigcontest2019/scripts/model/BJW/'

In [0]:
train_merge = pd.read_csv(path+'train.csv').fillna(0).reset_index(drop=True)
test1_merge = pd.read_csv(path+'test1.csv').fillna(0).reset_index(drop=True)
test2_merge = pd.read_csv(path+'test2.csv').fillna(0).reset_index(drop=True)

train_label = pd.read_csv('drive/My Drive/bigcontest2019/data/train/train_label.csv').reset_index(drop=True)

In [0]:
train_merge.

,acc_id,survival_time,amount_spent,trade_seller_day_count,trade_seller_type_nunique,trade_seller_type_personal,trade_seller_type_exchange,trade_seller_server_nunique,trade_seller_source_char_id_nunique,trade_seller_target_acc_id_nunique,trade_seller_target_char_id_nunique,trade_seller_item_type_nunique,trade_seller_item_amount_sum,trade_seller_item_price_sum,trade_buyer_day_count,trade_buyer_type_nunique,trade_buyer_type_personal,trade_buyer_type_exchange,trade_buyer_server_nunique,trade_buyer_target_char_id_nunique,trade_buyer_source_acc_id_nunique,trade_buyer_source_char_id_nunique,trade_buyer_item_type_nunique,trade_buyer_item_amount_sum,trade_buyer_item_price_sum,payment_day,payment_amount_spent,activity_day_nunique,activity_char_id_nunique,activity_server_nunique,activity_playtime_sum,activity_playtime_mean,activity_npc_kill_sum,activity_npc_kill_mean,activity_solo_exp_sum,activity_party_exp_sum,activity_quest_exp_sum,activity_rich_monster_sum,activity_death_sum,activity_revive_sum,activity_exp_recovery_sum,activity_fishing_sum,activity_private_shop_sum,activity_game_money_change_sum,activity_enchant_count_sum,combat_day,combat_char_id,combat_server,combat_class,combat_pledge_cnt,combat_random_attacker_cnt,combat_random_defender_cnt,combat_temp_cnt,combat_same_pledge_cnt,combat_etc_cnt,combat_num_opponent,pledge_day,pledge_char_id,pledge_server,pledge_pledge_id,pledge_play_char_cnt,pledge_combat_char_cnt,pledge_pledge_combat_cnt,pledge_random_attacker_cnt,pledge_random_defender_cnt,pledge_same_pledge_cnt,pledge_temp_cnt,pledge_etc_cnt,pledge_combat_play_time,pledge_non_combat_play_time
0,27835,64,0.002559,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,3.0,0.873062,26,3,1,9.854158,0.240345,12.769067,0.311441,0.130705,0.000000,0.022557,1,0.245883,0.247337,0.000000,1.184645,0.000000,0.151084,0.0,26,3,1,2,0.000000,0.0,0.000000,1.918911,0.000000,2.139619,1.962575,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,12351,64,0.120154,5.0,1.0,0.0,5.0,1.0,2.0,4.0,4.0,5.0,5.393214e+00,0.000000,20.0,2.0,14.0,6.0,1.0,2.0,15.0,15.0,6.0,1.294376,1.164605,1.0,0.528062,28,8,5,80.993692,1.079916,37.118665,0.494916,0.533964,0.039494,0.016483,7,0.983534,0.989348,0.000000,41.419533,0.000000,2.516748,0.0,28,8,5,6,7.124451,0.0,4.732258,5.756733,0.000000,2.364842,12.854867,28.0,5.0,5.0,5.0,89.681959,31.217261,31.072651,9.456912,6.151695,55.700204,50.133705,18.863577,104.079168,0.170040
2,125437,55,0.182593,18.0,1.0,0.0,18.0,1.0,1.0,16.0,16.0,3.0,2.908740e+00,0.000000,44.0,2.0,25.0,19.0,1.0,1.0,29.0,29.0,5.0,4.240387,2.444191,3.0,1.815358,27,3,1,39.552392,1.130068,13.483810,0.385252,0.176769,0.015612,0.134217,7,12.048290,12.119508,0.000000,23.852057,0.000000,5.731530,0.0,27,3,1,2,2.631554,0.0,2.366129,0.000000,0.000000,4.054015,6.770884,27.0,1.0,1.0,1.0,48.576224,28.618830,7.503624,18.106920,2.861254,39.785860,26.105125,34.202178,76.618756,0.000000
3,104483,64,0.016241,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,2.396984e-08,2.380465,20.0,2.0,18.0,2.0,1.0,1.0,13.0,13.0,3.0,0.431465,0.198727,1.0,0.371990,27,2,1,60.901508,1.268781,48.688721,1.014348,0.243562,0.001155,0.006941,2,1.475301,1.484021,31.897467,2.018737,0.114224,1.508932,0.0,27,2,1,1,0.000000,0.0,16.562903,1.918911,0.000000,2.139619,2.747605,18.0,2.0,1.0,3.0,72.142549,7.326132,0.305973,7.358962,2.575128,13.527192,20.567674,1.316557,37.404274,0.000000
4,4704,20,0.226396,8.0,1.0,0.0,8.0,1.0,1.0,6.0,6.0,4.0,5.153530e-02,0.000000,39.0,2.0,30.0,9.0,1.0,1.0,29.0,29.0,5.0,1.023133,5.527727,1.0,1.404644,28,6,5,25.679328,0.513587,17.134915,0.342698,0.433204,0.546727,0.227314,15,0.737650,0.742011,0.000000,9.309176,0.000000,0.454202,0.0,28,6,5,3,1.026948,0.0,3.549193,3.837822,12.398273,3.603569,5.298953,28.0,4.0,5.0,4.0,50.597226,20.137840,0.806822,3.518100,11.945734,8.752889,63.581800,13.080634,60.580907,0.003475


In [0]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
import lightgbm as lgb

In [0]:
FOLDS = 10
SEED = 42

kf = KFold(n_splits=FOLDS, random_state=SEED)

def rf_model(train,test,label, folds=FOLDS, seed=SEED):
    time_oof = np.zeros(len(train))
    spent_oof = np.zeros(len(train))

    time_pred = np.zeros(len(test))
    spent_pred = np.zeros(len(test))
    for trn_idx, val_idx in kf.split(train):
        train_df = train.loc[trn_idx]
        valid_df = train.loc[val_idx]
        
        rf_time = RandomForestRegressor().fit(X= train_df, y = label.loc[trn_idx, 'survival_time'])
        rf_spent = RandomForestRegressor().fit(X= train_df, y = label.loc[trn_idx, 'amount_spent'])
        
        time_oof[val_idx] = rf_time.predict(valid_df)
        time_pred += rf_time.predict(test)/kf.n_splits

        spent_oof[val_idx] = rf_spent.predict(valid_df)
        spent_pred += rf_spent.predict(test)/kf.n_splits

    return spent_oof, time_oof, spent_pred,time_pred

In [0]:
spent_oof_1, time_oof_1, spent_pred_1,time_pred_1 = rf_model(train_merge, test1_merge, train_label.sort_values('acc_id').reset_index(drop=True), FOLDS, SEED)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The de

In [0]:
# spent_oof_1[spent_oof_1<0] = 0
# time_oof_1[time_oof_1<0] = 0
# spent_pred_1[spent_pred_1<0] = 0
# time_pred_1[time_pred_1<0] = 0

In [0]:
# oof1_df = pd.DataFrame([np.unique(train_activity['acc_id']), time_oof_1.astype(int), spent_oof_1]).T
# oof1_df.columns = train_label.columns
# oof1_df[['acc_id', 'survival_time']] = oof1_df[['acc_id', 'survival_time']].astype(int)

In [0]:
# oof1_df

In [0]:
# path = 'drive/My Drive/bigcontest2019/scripts'
# os.chdir(path)
# from model.metrics import score_function

In [0]:
# oof1_df2 = oof1_df.copy()
# # oof1_df2['amount_spent'] = oof1_df2['amount_spent']*70
# # oof1_df2.loc[(oof1_df2['survival_time']>7) & (oof1_df2['survival_time']<64), 'survival_time'] = 35
# score_function.score_function(oof1_df2, train_label, path=False)

13675.722421886228


13675.722421886228

In [0]:
# pred1_df = pd.DataFrame([np.unique(test1_activity['acc_id']), pred1_time.astype(int), pred1_spent]).T
# pred1_df.columns = train_label.columns
# pred1_df[['acc_id', 'survival_time']] = pred1_df[['acc_id', 'survival_time']].astype(int)

# pred1_df.loc[pred1_df['survival_time']<1, 'survival_time'] = 1
# pred1_df.loc[pred1_df['survival_time']>64, 'survival_time'] = 64

# # pred1_df.loc[(pred1_df['survival_time']>7) & (pred1_df['survival_time']<64), 'survival_time'] = 30
# pred1_df.loc[pred1_df['amount_spent']<0, 'amount_spent']=0

# pred1_df['amount_spent'] = pred1_df['amount_spent']

# pred1_df.to_csv('/content/test1_predict.csv', index=False, encoding='utf-8')

In [0]:
FOLDS=5;SEED=42
def lgb_model(train, test, label, folds=FOLDS, seed=SEED):
    
    params = {
        'objective':'regression',
        "boosting": "gbdt",
        "num_iterations ":100,
        'num_leaves': 10000,
        'max_depth': 8,
        'learning_rate': 0.1,
        'min_data_in_leaf': 32, 
        'min_child_samples': 30,
        'min_child_weight': 0.5,
        'min_split_gain': 0.005,
        "feature_fraction": 0.9,
        "bagging_fraction": 0.9 ,
        "bagging_freq": 2,
        "bagging_seed": 42,
        "metric": 'rmse',
        "lambda_l1": 0.1,
        "lambda_l2": 0.1
    }
    
    skf = StratifiedKFold(n_splits=folds, random_state=seed)
    kf = KFold(n_splits=folds, random_state=seed)
    
    oof_time = np.zeros(len(train))
    oof_spent = np.zeros(len(train))
    pred_time = np.zeros(len(test))
    pred_spent = np.zeros(len(test))
    
    for trn_idx, val_idx in kf.split(train):
        
        train_df = lgb.Dataset(train.loc[trn_idx], label=label.loc[trn_idx, 'amount_spent'])
        valid_df = lgb.Dataset(train.loc[val_idx], label=label.loc[val_idx, 'amount_spent'])
        
        lgb_model = lgb.train(params, train_df, 5000, valid_sets = [train_df, valid_df], early_stopping_rounds = 500, verbose_eval=5000)
        oof_spent[val_idx] = lgb_model.predict(train.loc[val_idx])/skf.n_splits
        pred_spent += lgb_model.predict(test)/kf.n_splits
    
    params = {
        'objective':'multiclass',
        'num_class':65,
        "boosting": "gbdt",
        "num_iterations ":100,
        'num_leaves': 10000,
        'max_depth': 8,
        'learning_rate': 0.1,
        'min_data_in_leaf': 32, 
        'min_child_samples': 30,
        'min_child_weight': 0.5,
        'min_split_gain': 0.005,
        "feature_fraction": 0.9,
        "bagging_fraction": 0.9 ,
        "bagging_freq": 2,
        "bagging_seed": 42,
        "metric": 'multi_logloss',
        "lambda_l1": 0.1,
        "lambda_l2": 0.1
    }
    train['amount_spent'] = oof_time
    for trn_idx, val_idx in skf.split(train, label['survival_time']):
        
        train_df = lgb.Dataset(train.loc[trn_idx], label=label.loc[trn_idx, 'survival_time'])
        valid_df = lgb.Dataset(train.loc[val_idx], label=label.loc[val_idx, 'survival_time'])
        
        lgb_model = lgb.train(params, train_df, 5000, valid_sets = [train_df, valid_df], early_stopping_rounds = 500, verbose_eval=5000)
        oof_time[val_idx] = lgb_model.predict(train.loc[val_idx])/skf.n_splits
        pred_time += lgb_model.predict(test)/skf.n_splits
    
    return oof_time, oof_spent, pred_time, pred_spent

In [0]:
oof1_time, oof1_spent, pred1_time, pred1_spent = lgb_model(
    train_merge, test1_merge, train_label.sort_values('acc_id').reset_index(drop=True), FOLDS, SEED)

Training until validation scores don't improve for 500 rounds.
Early stopping, best iteration is:
[18]	training's rmse: 0.705659	valid_1's rmse: 0.543462
Training until validation scores don't improve for 500 rounds.
Early stopping, best iteration is:
[26]	training's rmse: 0.641854	valid_1's rmse: 0.77594
Training until validation scores don't improve for 500 rounds.
Early stopping, best iteration is:
[7]	training's rmse: 0.706828	valid_1's rmse: 0.648137
Training until validation scores don't improve for 500 rounds.
Early stopping, best iteration is:
[28]	training's rmse: 0.561883	valid_1's rmse: 0.993503
Training until validation scores don't improve for 500 rounds.
Early stopping, best iteration is:
[7]	training's rmse: 0.741277	valid_1's rmse: 0.462554
Training until validation scores don't improve for 500 rounds.
Early stopping, best iteration is:
[37]	training's multi_logloss: 0.948987	valid_1's multi_logloss: 1.96714


ValueError: ignored

In [0]:
oof1_df = pd.DataFrame([np.unique(train_activity['acc_id']), oof1_time.astype(int), oof1_spent]).T
oof1_df.columns = train_label.columns
oof1_df[['acc_id', 'survival_time']] = oof1_df[['acc_id', 'survival_time']].astype(int)

In [0]:
path = 'drive/My Drive/bigcontest2019/scripts'
os.chdir(path)
from model.metrics import score_function

In [0]:
oof1_df2 = oof1_df.copy()
# oof1_df2['amount_spent'] = oof1_df2['amount_spent']*70
# oof1_df2.loc[(oof1_df2['survival_time']>7) & (oof1_df2['survival_time']<64), 'survival_time'] = 35
score_function.score_function(oof1_df2, train_label, path=False)

29107.194324085198


29107.194324085198

In [0]:
score_function.score_function(train_label2, train_label, path=False)

56281.93961878003


56281.93961878003

In [0]:
pred1_df = pd.DataFrame([np.unique(test1_activity['acc_id']), pred1_time.astype(int), pred1_spent]).T
pred1_df.columns = train_label.columns
pred1_df[['acc_id', 'survival_time']] = pred1_df[['acc_id', 'survival_time']].astype(int)

pred1_df.loc[pred1_df['survival_time']<1, 'survival_time'] = 1
pred1_df.loc[pred1_df['survival_time']>64, 'survival_time'] = 64

# pred1_df.loc[(pred1_df['survival_time']>7) & (pred1_df['survival_time']<64), 'survival_time'] = 30
pred1_df.loc[pred1_df['amount_spent']<0, 'amount_spent']=0

pred1_df['amount_spent'] = pred1_df['amount_spent']

pred1_df.to_csv('/content/test1_predict.csv', index=False, encoding='utf-8')

In [0]:
def xgb_f1_score(y_hat, data):
    y_true = data.get_label()
    y_hat = custom_round(y_hat, np.quantile(y_hat, 0.85))
    return 'f1', -f1_score(y_true, y_hat)

params = {
    'objective':'binary:logistic',
    'n_estimators':10000,
    'max_depth':16,
    'learning_rate':0.001,
    'subsample':0.7,
    'colsample_bytree':0.7,
    'reg_alpha':0.2,
    'tree_method':'gpu_hist',
    'seed':42
}

skf = StratifiedKFold(n_splits=5, random_state=42)

pred = np.zeros(len(test_df))

for trn_idx, val_idx in skf.split(train_df, train_label['label']):
    
    trn_df = xgb.DMatrix(train_df.loc[trn_idx], label=train_label.loc[trn_idx, 'label'])
    val_df = xgb.DMatrix(train_df.loc[val_idx], label=train_label.loc[val_idx, 'label'])
    te_df = xgb.DMatrix(test_df)
    
    xgb_model = xgb.train(params, trn_df, num_boost_round=5000, 
                          evals=[(trn_df, 'train'), (val_df, 'val')], 
                          early_stopping_rounds = 10, verbose_eval=100, feval=xgb_f1_score)
    pred += xgb_model.predict(te_df)/skf.n_splits

In [0]:
pred2_time, pred2_spent = lgb_model(train_activity_merge, test2_activity_merge, train_label.sort_values('acc_id').reset_index(drop=True), FOLDS, SEED)

Training until validation scores don't improve for 500 rounds.
Early stopping, best iteration is:
[595]	training's rmse: 13.0762	valid_1's rmse: 17.8931
Training until validation scores don't improve for 500 rounds.
Early stopping, best iteration is:
[349]	training's rmse: 13.8865	valid_1's rmse: 18.313
Training until validation scores don't improve for 500 rounds.
Early stopping, best iteration is:
[420]	training's rmse: 13.6981	valid_1's rmse: 17.8818
Training until validation scores don't improve for 500 rounds.
Early stopping, best iteration is:
[764]	training's rmse: 12.6094	valid_1's rmse: 17.8254
Training until validation scores don't improve for 500 rounds.
Early stopping, best iteration is:
[506]	training's rmse: 13.324	valid_1's rmse: 18.1785
Training until validation scores don't improve for 500 rounds.
Early stopping, best iteration is:
[114]	training's rmse: 0.712953	valid_1's rmse: 0.544113
Training until validation scores don't improve for 500 rounds.
Early stopping, bes

In [0]:
pred1_df = pd.DataFrame([np.unique(test1_activity['acc_id']), pred1_time.astype(int), pred1_spent]).T
pred1_df.columns = train_label.columns
pred1_df[['acc_id', 'survival_time']] = pred1_df[['acc_id', 'survival_time']].astype(int)

pred2_df = pd.DataFrame([np.unique(test2_activity['acc_id']), pred2_time.astype(int), pred2_spent]).T
pred2_df.columns = train_label.columns
pred2_df[['acc_id', 'survival_time']] = pred2_df[['acc_id', 'survival_time']].astype(int)

In [0]:
pred1_df.loc[pred1_df['survival_time']<1, 'survival_time'] = 1
pred2_df.loc[pred2_df['survival_time']<1, 'survival_time'] = 1
pred1_df.loc[pred1_df['survival_time']>64, 'survival_time'] = 64
pred2_df.loc[pred2_df['survival_time']>64, 'survival_time'] = 64

pred1_df.loc[(pred1_df['survival_time']>7) & (pred1_df['survival_time']<64), 'survival_time'] = 30
pred2_df.loc[(pred2_df['survival_time']>7) & (pred2_df['survival_time']<64), 'survival_time'] = 30

pred1_df.loc[pred1_df['amount_spent']<0, 'amount_spent']=0
pred2_df.loc[pred2_df['amount_spent']<0, 'amount_spent']=0

pred1_df['amount_spent'] = pred1_df['amount_spent']*20
pred2_df['amount_spent'] = pred2_df['amount_spent']*20

pred1_df.to_csv('/content/test1_predict.csv', index=False, encoding='utf-8')
pred2_df.to_csv('/content/test2_predict.csv', index=False, encoding='utf-8')

# train, test split

In [0]:
skf = StratifiedKFold(n_splits=5, random_state=SEED, shuffle=True)
for trn_idx, val_idx in skf.split(train_label, y=train_label['survival_time']):
    train_df = train_label.loc[trn_idx]
    valid_df = train_label.loc[val_idx]
    break

# modeling

In [0]:
trn_activity = train_activity.loc[train_activity['acc_id'].isin(train_df['acc_id']), ['day', 'acc_id', 'playtime']]
val_activity = train_activity.loc[train_activity['acc_id'].isin(valid_df['acc_id']), ['day', 'acc_id', 'playtime']]

In [0]:
trn_activity = trn_activity.groupby(['acc_id', 'day']).sum().unstack()
val_activity = val_activity.groupby(['acc_id', 'day']).sum().unstack()

In [0]:
trn_activity.columns = trn_activity.columns.droplevel(0)
val_activity.columns = val_activity.columns.droplevel(0)

In [0]:
trn_activity = trn_activity.reset_index()
val_activity = val_activity.reset_index()

In [0]:
train_df = train_df.reset_index(drop=True)
valid_df = valid_df.reset_index(drop=True)
train_df = pd.merge(train_df, trn_activity.fillna(0), how='left', on='acc_id')
valid_df = pd.merge(valid_df, val_activity.fillna(0), how='left', on='acc_id')

In [0]:
rf_survival = RandomForestRegressor(n_estimators=500, random_state=SEED)
rf_spent = RandomForestRegressor(n_estimators=500, random_state=SEED)

In [0]:
rf_survival.fit(train_df.iloc[:, 3:], train_df['survival_time'])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=500,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [0]:
rf_spent.fit(train_df.iloc[:, 3:], train_df['amount_spent'])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=500,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

# inference

In [0]:
survival_inference = rf_survival.predict(valid_df.iloc[:, 3:])
spent_inference = rf_spent.predict(valid_df.iloc[:, 3:])

In [0]:
survival_inference = np.round(survival_inference, 0)
predict_df = pd.DataFrame(np.concatenate([valid_df['acc_id'].values.reshape(-1, 1), survival_inference.reshape(-1, 1), spent_inference.reshape(-1, 1)], axis=1), columns=valid_df.columns[:3])

In [0]:
output_path = 'drive/My Drive/bigcontest2019/metrics/lhh_inference/'

In [0]:
valid_df.iloc[:, :3].to_csv(output_path+'true.csv', index=False)
predict_df.to_csv(output_path+'predict.csv', index=False)